# Clasificación de textos

En esta práctica vamos a construir un clasificador de textos utilizando el módulo `scikit-learn`. Vamos a trabajar con textos relacionados con los temas de medicina y aeronaútica y vamos a ver si nuestro modelo es capaz de clasificarlos e incluso de aprender a hacerlo con textos que nosotros mismos le pasemos.

## Carga de datos

Comenzamos cargando los datos que vamos a utilizar. En este caso va a ser un dataset llamado "20 Newsgroups" que es un conjunto formado por documentos de texto agrupados en 20 categorías distintas correspondiéndose cada categoría a un tema. En nuestro caso nos vamos a quedar solo con dos temas concretos: medicina y aerospacial.

El conjunto de datos se encuentra entre los datos de muestra de `scikit-learn`.

Veamos cómo cargar los datos:

In [2]:
from sklearn.datasets import fetch_20newsgroups
datos = fetch_20newsgroups(subset='train', categories=['sci.med', 'sci.space'])

Estamos cargando únicamente los datos de entrenamiento de las categorías medicina y espacio. Echemos un ojo a los datos:



In [3]:
datos.data[2]

"From: ab961@Freenet.carleton.ca (Robert Allison)\nSubject: Frequent nosebleeds\nReply-To: ab961@Freenet.carleton.ca (Robert Allison)\nOrganization: The National Capital Freenet\nLines: 18\n\n\nI have between 15 and 25 nosebleeds each week, as a result of a genetic\npredisposition to weak capillary walls (Osler-Weber-Rendu). Fortunately,\neach nosebleed is of short duration.\n\nDoes anyone know of any method to reduce this frequency? My younger brothers\neach tried a skin transplant (thigh to nose lining), but their nosebleeds\nsoon returned. I've seen a reference to an herb called Rutin that is\nsupposed to help, and I'd like to hear of experiences with it, or other\ntechniques.\n-- \nRobert Allison\nOttawa, Ontario CANADA\n"

In [4]:
datos.target[2]

0

In [5]:
datos.data[38]

'From: dennisn@ecs.comm.mot.com (Dennis Newkirk)\nSubject: Re: First Spacewalk\nOrganization: Motorola\nDistribution: sci\nNntp-Posting-Host: 145.1.146.43\nLines: 14\n\nIn article <C5suMG.2rF.1@cs.cmu.edu> flb@flb.optiplan.fi ("F.Baube[tm]") writes:\n>At one time there was speculation that the first spacewalk \n>(Alexei Leonov ?) was a staged fake.\n>\n>Has any evidence to support or contradict this claim emerged ?\n>\n>Was this claim perhaps another fevered Cold War hallucination ?\n\nThis claim was made when someone spotted training film footage spliced into\nthe footage of the actual spacewalk.\n\nDennis Newkirk (dennisn@ecs.comm.mot.com)\nMotorola, Land Mobile Products Sector\nSchaumburg, IL\n'

In [6]:
datos.target[38]

1

## Construcción del modelo

Vamos a construir nuestro modelo a la vez que el preprocesamiento, esto nos permitirá tener el proceso agrupado en una única "pipeline" y que posteriormente podamos pasar un texto normal y la misma pipeline lo preprocese y realice la inferencia. Para ello la pipeline primero usará `CountVectorizer`que convierte el texto en una matriz de recuento de palabras y a continuación `MultinomialNB` que implementa un clasificador de Bayes:

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

modelo = make_pipeline(CountVectorizer(), MultinomialNB())

Una vez creada la pipeline del modelo podemos proceder a entrenarlo sobre los datos:

In [8]:
modelo.fit(datos.data, datos.target)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('multinomialnb', MultinomialNB())])

¡Ya tenemos nuestro modelo entrenado! Vamos a comprobar qué tal funciona:

In [9]:
# Cargar el conjunto de datos de validación
datos_validacion = fetch_20newsgroups(subset='test', categories=['sci.med', 'sci.space'])

# Evaluar el modelo en el conjunto de validación
accuracy = modelo.score(datos_validacion.data, datos_validacion.target)

print(f'Tasa de acierto en validación: {accuracy}')


Tasa de acierto en validación: 0.9822784810126582


El clasificador es súper bueno, acierta en un 98% de los casos. Podemos estudiar sus métricas más en profundidad:

In [10]:
from sklearn.metrics import classification_report

# Predecir las etiquetas en el conjunto de validación
predicciones = modelo.predict(datos_validacion.data)

# Generar el informe de clasificación
report = classification_report(datos_validacion.target, predicciones, target_names=datos_validacion.target_names)

# Imprimir el informe de clasificación
print(report)

              precision    recall  f1-score   support

     sci.med       0.98      0.98      0.98       396
   sci.space       0.98      0.98      0.98       394

    accuracy                           0.98       790
   macro avg       0.98      0.98      0.98       790
weighted avg       0.98      0.98      0.98       790



Vemos que las clases están equilibradas y que por tanto la tasa de acierto es métrica sufiente para saber que es muy bueno modelo.

Por último podemos probarlo con nuestras propias frases:

In [11]:
etiquetas = modelo.predict(['10 satellites were launched yesterday', 'Researchers discover new vaccine for flu'])
for etiqueta in etiquetas:
    print(datos.target_names[etiqueta])

sci.space
sci.med


## Conclusiones

Solo con unas pocas líneas de código hemos conseguido implementar un clasificador que da unos resultados extraordinarios. Os animo a que exploréis la clasificación de otras categorías; os dejo debajo el listado de todas las que existen. Además podéis probar el modelo introduciendo vuestras propias frases.

In [12]:
datos_todas_categorias = fetch_20newsgroups()

In [13]:
datos_todas_categorias.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']